In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
root = '../../resources'

In [91]:
def clean_cd(data):
  data.sort_values(['number', 'time'], inplace=True)
#   data = data.assign(usage=lambda x: (x.total-x.available)/x.total)
  nd = pd.DataFrame(data.number.value_counts())
  nd = nd[nd.number<=288]
  data = data[data.number.isin(nd.index)]
  data.time = pd.to_datetime(data.time)
  data.time = data.time.dt.hour
  data = data.groupby(['number', 'time']).mean()
  return data.reset_index().assign(n=1)

In [ ]:
jeff = pd.read_csv(f'{root}/2020/2020-12-31.csv')
jeff = clean_cd(jeff)
jeff

In [128]:
def mean_year(year):
    td = timedelta(days=1)
    dt = datetime.strptime(f'{year}-01-01', '%Y-%m-%d')
    dend = datetime.strptime(f'{year}-12-31', '%Y-%m-%d')
    frick = pd.DataFrame()
    while dt <= dend:
        bruh = dt.strftime('%Y-%m-%d')
        name = f'{bruh}.csv'
        jeff = pd.read_csv(f'{root}/{year}/{name}')
        if frick.empty:
            frick = clean_cd(jeff)
        else:
            jeff = clean_cd(jeff)
            frick = frick.append(jeff).groupby(['number', 'time']).sum().reset_index()
            # frick.total += jeff.total
            # frick.available += jeff.available
        dt += td
    frick.total /= frick.n
    frick.available /= frick.n
    frick.to_csv(f'{root}/meanyear/{year}.csv', index=False)

In [129]:
for i in ['2020', '2019', '2018']:
    mean_year(i)

In [131]:
jeff = pd.read_csv(f'{root}/meanyear/2020.csv')
jeff.value_counts()

number  time  total       available   n  
A10     0     63.525140   17.407123   358    1
SI8     19    118.000000  79.506446   362    1
SI6     3     40.340782   9.599860    358    1
        4     40.340782   9.533287    358    1
        5     40.337047   9.032962    359    1
                                            ..
HG50    17    469.265193  242.158149  362    1
        18    468.812845  223.257366  362    1
        19    468.179558  197.417818  362    1
        20    468.179558  175.313306  362    1
YHS     23    60.507202   21.052533   251    1
Length: 45264, dtype: int64